In [1]:
from crypto_pandas import CCXTPandasExchange
import ccxt
import pandas as pd

In [2]:
# Setup exchanges
binance = CCXTPandasExchange(exchange=ccxt.binance())
bybit = CCXTPandasExchange(exchange=ccxt.bybit())
kucoin = CCXTPandasExchange(ccxt.kucoinfutures())

In [3]:
# Fetch bids and asks
binance_ba = binance.fetch_bids_asks()
bybit_ba = bybit.fetch_bids_asks()
kucoin_ba = kucoin.fetch_bids_asks()

# Combine and display
data = pd.concat([
    binance_ba,
    bybit_ba,
    kucoin_ba,
])

data.dropna(axis=1)

,symbol,bidVolume,askVolume,exchange
0,ETH/BTC,49.028,6.4421,binance
1,LTC/BTC,6.125,333.5320,binance
2,BNB/BTC,4.066,1.1800,binance
3,NEO/BTC,22.430,1726.4700,binance
4,QTUM/ETH,4529.400,42.5000,binance
...,...,...,...,...
453,AR/USDT:USDT,132.000,80.0000,kucoinfutures
454,B3/USDT:USDT,2715.000,3776.0000,kucoinfutures
455,PNUT/USDT:USDT,2533.000,3902.0000,kucoinfutures
456,SYRUP/USDT:USDT,298.000,20.0000,kucoinfutures


In [5]:
# Find all combinations and calculate spread
arbitrage = data[["exchange", "symbol", "bid"]].merge(data[["exchange", "symbol", "ask"]], on="symbol", suffixes=("Short", "Long"))
arbitrage["spread"] = arbitrage["bid"] - arbitrage["ask"]
arbitrage["spreadRelative"] = arbitrage["spread"]/arbitrage[["bid", "ask"]].mean(axis=1)

# Filter for different exchanges for buy/sell
arbitrage = arbitrage.query("exchangeShort != exchangeLong").sort_values("spreadRelative", ascending=False, ignore_index=True)
arbitrage.head(10)

,exchangeShort,symbol,bid,exchangeLong,ask,spread,spreadRelative
0,bybit,SIREN/USDT:USDT,0.05868,kucoinfutures,0.058380,0.000300,0.005126
1,bybit,ASP/USDT:USDT,0.21110,kucoinfutures,0.210400,0.000700,0.003321
2,bybit,RFC/USDT:USDT,0.00669,kucoinfutures,0.006670,0.000020,0.002994
3,bybit,GORK/USDT:USDT,0.00795,kucoinfutures,0.007927,0.000023,0.002897
4,bybit,DODO/USDT:USDT,0.04881,kucoinfutures,0.048690,0.000120,0.002462
5,kucoinfutures,SONIC/USDT:USDT,0.24670,bybit,0.246100,0.000600,0.002435
6,bybit,BAN/USDT:USDT,0.06474,kucoinfutures,0.064610,0.000130,0.002010
7,bybit,KOMA/USDT:USDT,0.02538,kucoinfutures,0.025330,0.000050,0.001972
8,bybit,WOO/USDT:USDT,0.07825,kucoinfutures,0.078100,0.000150,0.001919
9,kucoinfutures,J/USDT:USDT,0.10570,bybit,0.105500,0.000200,0.001894
